In [10]:
from safetensors.torch import load_file
import torch

path = "/home/parkjeongsu/TinyVLA/Llava-Pythia-700M/model.safetensors"
weights = load_file(path)

print("✅ 전체 키 목록:")
for k in weights.keys():
    print(k)

print("\n✅ 주요 가중치 분석:")
combine_weight = weights.get("embed_out.combine.weight", None)
if combine_weight is not None:
    print("🧩 combine.weight shape:", combine_weight.shape)
    print("📈 mean:", combine_weight.mean().item())
    print("📉 std:", combine_weight.std().item())
    print("✅ norm:", combine_weight.norm().item())
else:
    print("❌ combine.weight 없음")

# gripper head 쪽 확인
if "embed_out.action_head.weight" in weights:
    gripper_head = weights["embed_out.action_head.weight"][-1]  # 마지막 차원이 gripper라고 가정
    print("🤖 gripper head mean/std:", gripper_head.mean().item(), "/", gripper_head.std().item())


✅ 전체 키 목록:
embed_out.weight
gpt_neox.embed_in.weight
gpt_neox.final_layer_norm.bias
gpt_neox.final_layer_norm.weight
gpt_neox.layers.0.attention.dense.bias
gpt_neox.layers.0.attention.dense.weight
gpt_neox.layers.0.attention.query_key_value.bias
gpt_neox.layers.0.attention.query_key_value.weight
gpt_neox.layers.0.input_layernorm.bias
gpt_neox.layers.0.input_layernorm.weight
gpt_neox.layers.0.mlp.dense_4h_to_h.bias
gpt_neox.layers.0.mlp.dense_4h_to_h.weight
gpt_neox.layers.0.mlp.dense_h_to_4h.bias
gpt_neox.layers.0.mlp.dense_h_to_4h.weight
gpt_neox.layers.0.post_attention_layernorm.bias
gpt_neox.layers.0.post_attention_layernorm.weight
gpt_neox.layers.1.attention.dense.bias
gpt_neox.layers.1.attention.dense.weight
gpt_neox.layers.1.attention.query_key_value.bias
gpt_neox.layers.1.attention.query_key_value.weight
gpt_neox.layers.1.input_layernorm.bias
gpt_neox.layers.1.input_layernorm.weight
gpt_neox.layers.1.mlp.dense_4h_to_h.bias
gpt_neox.layers.1.mlp.dense_4h_to_h.weight
gpt_neox.laye

In [3]:
import pickle

def load_pkl(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

# 사용 예시
file_path = "/home/parkjeongsu/TinyVLA/OUTPUT_llava_pythia/dataset_stats.pkl"
data = load_pkl(file_path)

# 상위 키 출력
if isinstance(data, dict):
    print("🔑 Top-level keys:", list(data.keys())[:10])

# 예시로 일부 값 출력
for key in list(data.keys())[:2]:
    print(f"\n📌 [{key}] 값:")
    print(data[key])


🔑 Top-level keys: ['action_mean', 'action_std', 'action_min', 'action_max', 'qpos_mean', 'qpos_std', 'example_qpos']

📌 [action_mean] 값:
[ 0.5505599   0.015884    0.28063768  0.768541   -0.02702801 -0.06607575
 -0.03195749 -0.79086196 -0.02463053  0.34631187]

📌 [action_std] 값:
[0.11837748 0.17448317 0.15690881 0.34308028 0.46564147 0.26412168
 0.4723583  0.3456244  0.17425872 0.41214043]


In [ ]:
# 역정규화에 필요한 값
ACTION_MEAN = torch.tensor([
    0.5505599, 0.015884, 0.28063768, 0.768541, -0.02702801,
    -0.06607575, -0.03195749, -0.79086196, -0.02463053, 0.34631187
]).cuda()

ACTION_STD = torch.tensor([
    0.11837748, 0.17448317, 0.15690881, 0.34308028, 0.46564147,
    0.26412168, 0.4723583, 0.3456244, 0.17425872, 0.41214043
]).cuda()
